#  Word2vec

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://radimrehurek.com/gensim/models/word2vec.html
* https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html
* https://pytorch.org/text/stable/vocab.html
* https://github.com/OlgaChernytska/word2vec-pytorch
* https://www.baeldung.com/cs/nlps-word2vec-negative-sampling
* https://towardsdatascience.com/implementing-word2vec-in-pytorch-from-the-ground-up-c7fe5bf99889

## Задачи для совместного разбора

1\. Рассмотрите основные шаги подготовки данных для обучения skip-gram модели

In [ ]:
text = "Спящий котик мило моргает своими яркими глазками"

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
corpus = [
    text.lower().split()
]
vocab = build_vocab_from_iterator(corpus)

In [ ]:
corpus_i = [
    vocab.lookup_indices(t)
    for t in corpus
]
corpus_i

[[5, 1, 2, 3, 4, 6, 0]]

In [ ]:
ex = corpus_i[0]
input, output = [], []
for idx, word in enumerate(ex):
  if idx == 0 or idx == len(ex) - 1:
    continue
  input.append(word)
  output.append(ex[idx - 1])
  input.append(word)
  output.append(ex[idx + 1])

In [ ]:
import torch as th

input = th.tensor(input, dtype=th.long)
output = th.tensor(output, dtype=th.long)

2\. Рассмотрите основные шаги по настройке skip-gram модели

In [ ]:
import torch.nn as nn

emb = nn.Embedding(len(vocab), 300)

In [ ]:
len(input), emb(input).shape

(10, torch.Size([10, 300]))

In [ ]:
fc = nn.Linear(in_features=300, out_features=len(vocab))

In [ ]:
x_e = emb(input)
out = fc(x_e)
out.shape

torch.Size([10, 7])

In [ ]:
model = nn.Sequential(
    nn.Embedding(len(vocab), 300),
    nn.Linear(in_features=300, out_features=len(vocab))
)
crit = nn.CrossEntropyLoss()
out = model(input)
loss = crit(out, output)
loss

tensor(2.0615, grad_fn=<NllLossBackward0>)

In [ ]:
emb = nn.Embedding(len(vocab), 16, max_norm=1)
inputs_e = emb(input)
outputs_e = emb(output)

In [ ]:
inputs_e[0] @ outputs_e[0]

tensor(0.0706, grad_fn=<DotBackward0>)

In [ ]:
(inputs_e @ outputs_e.T).shape

torch.Size([10, 10])

In [ ]:
o = inputs_e.view(-1, 1, 16).bmm(outputs_e.view(-1, 16, 1))
o

tensor([[[ 0.0706]],

        [[ 0.0801]],

        [[ 0.0801]],

        [[-0.1142]],

        [[-0.1142]],

        [[ 0.0977]],

        [[ 0.0977]],

        [[-0.4377]],

        [[-0.4377]],

        [[-0.1486]]], grad_fn=<BmmBackward0>)

## Задачи для самостоятельного решения

In [ ]:
import pandas as pd
import nltk
import pymorphy2
from nltk import sent_tokenize, RegexpTokenizer, word_tokenize
import re
from nltk.corpus import stopwords
from torchtext.vocab import build_vocab_from_iterator


nltk.download('punkt')
nltk.download('stopwords')
sw = stopwords.words('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<p class="task" id="1"></p>

1\. Загрузите тексты новостей из файла `news_500.csv`. Удалите из текстов все знаки препинания и символы не из русского алфавита, приведите все слова к нижнему регистру и удалите стоп-слова. Разбейте текст каждой новости, удалив из них стоп-слова. Разбейте текст каждой новости на фрагменты по 3 предложения и сохраните в виде списка строк. Выведите на экран длину полученного списка.

- [x] Проверено на семинаре

In [ ]:
def processing(text):
  tokenizer = RegexpTokenizer(r'\w+')
  sents = sent_tokenize(text)
  sents_split = [sents[i:i+3] for i in range(0, len(sents), 3)]
  res = []
  for s in sents_split:
    words = []
    s_buf = ' '.join(s).lower()
    s_pure = tokenizer.tokenize(re.sub(r'[^А-Яа-я ]', ' ', s_buf))
    for word in s_pure:
      if word not in sw:
        words.append(word)
    res.append(' '.join(words))
  return res

In [ ]:
df = pd.read_csv('news_500.csv', usecols=['text'])
proc = df['text'].apply(processing)
corpus = proc.sum()

In [ ]:
len(corpus)

1965

<p class="task" id="2"></p>

2\. Настройте модель Word2Vec из пакета `gensim`. Для валидации выведите на экран информацию о ближайших словах для нескольких случайно выбранных токенов из обучающей выборки.

- [ ] Проверено на семинаре

In [ ]:
corpus_t = [word_tokenize(s) for s in corpus]

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpus_t, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
from random import randint
rand_words = []
for _ in range(2):
  ind1 = randint(0, len(corpus_t)-1)
  ind2 = randint(0, len(corpus_t[ind1])-1)
  rand_words.append(corpus_t[ind1][ind2])
rand_words

['могло', 'дефицита']

In [ ]:
for word in rand_words:
  print(f'{word}\n{model.wv.most_similar(word, topn=5)}')

могло
[('информации', 0.6751124262809753), ('г', 0.6723748445510864), ('составляет', 0.6688089370727539), ('около', 0.6651775240898132), ('недели', 0.6649465560913086)]
дефицита
[('валютным', 0.4208981394767761), ('суриков', 0.42014119029045105), ('основной', 0.39718472957611084), ('канадских', 0.38568130135536194), ('афганистан', 0.38430055975914)]


<p class="task" id="3"></p>

3\. Опишите класс `W2VDataset`, который реализует в себе логику получения контекстного окна для обучения skip-gram модели. При создании словаря игнорируйте токены, которые встретились меньше 20 раз. Продемонстрируйте пример работы.

![image.png](attachment:image.png)

- [ ] Проверено на семинаре

In [ ]:
class W2VDataset():
  def __init__(self, corpus):
    corpus_t = [word_tokenize(s) for s in corpus]
    self.vocab = build_vocab_from_iterator(
        corpus_t, min_freq=20, specials=['<PAD>', '<UNK>']
    )
    self.vocab.set_default_index(self.vocab['<UNK>'])

  def context(self, batch, window=3):
    input, output = [], []
    gap = window // 2
    batch_i = [
        self.vocab.lookup_indices(tokens)
        for tokens in batch
    ]
    for corpus_i in batch_i:
      for idx, word in enumerate(corpus_i):
        if idx < gap or idx > len(corpus_i) - gap - 1:
          continue
        input.extend((window - 1)*[word])
        output.extend(corpus_i[idx - gap: idx])
        output.extend(corpus_i[idx + 1: idx + gap + 1])
    return input, output

In [ ]:
dataset = W2VDataset(corpus)
corpus_t = [word_tokenize(s) for s in corpus]
dataset.context([corpus_t[0]], window=3)

([3,
  3,
  54,
  54,
  125,
  125,
  36,
  36,
  1,
  1,
  1,
  1,
  1,
  1,
  154,
  154,
  1,
  1,
  1,
  1,
  173,
  173,
  320,
  320,
  1,
  1,
  1,
  1,
  348,
  348,
  49,
  49,
  73,
  73,
  4,
  4,
  20,
  20,
  2,
  2,
  104,
  104,
  33,
  33,
  125,
  125,
  5,
  5,
  12,
  12,
  424,
  424,
  1,
  1,
  154,
  154,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  118,
  118,
  11,
  11,
  1,
  1,
  1,
  1],
 [50,
  54,
  3,
  125,
  54,
  36,
  125,
  1,
  36,
  1,
  1,
  1,
  1,
  154,
  1,
  1,
  154,
  1,
  1,
  173,
  1,
  320,
  173,
  1,
  320,
  1,
  1,
  348,
  1,
  49,
  348,
  73,
  49,
  4,
  73,
  20,
  4,
  2,
  20,
  104,
  2,
  33,
  104,
  125,
  33,
  5,
  125,
  12,
  5,
  424,
  12,
  1,
  424,
  154,
  1,
  1,
  154,
  1,
  1,
  1,
  1,
  1,
  1,
  118,
  1,
  11,
  118,
  1,
  11,
  1,
  1,
  4])

<p class="task" id="4"></p>

4\. Реализуйте и настройте skip-gram модель. Перед началом обучения выберите случайным образом несколько слов из датасета и для каждого из них выведите на экран 3 ближайших слова в смысле косинусной близости между эмбеддингами. В процессе настройки для валидации периодически выводите на экран информацию о ближайших словах для этих слов. Выведите на экран график значения функции потерь в зависимости от номера эпохи.  

![image.png](attachment:image.png)

- [ ] Проверено на семинаре

In [ ]:
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader

In [ ]:
class SkipGramModel(nn.Module):
  def __init__(self, vocab_len):
    super().__init__()
    self.emb = nn.Embedding(
        num_embeddings=vocab_len,
        embedding_dim=300,
        padding_idx=0
    )
    self.fc = nn.Linear(in_features=300, out_features=vocab_len)

  def forward(self, X):
    e = self.emb(X)
    out = self.fc(e)
    return out

In [ ]:
dataset = W2VDataset(corpus)
X, y = dataset.context(corpus_t)
X = th.tensor(X).long()
y = th.tensor(y).long()
X.shape, y.shape

(torch.Size([137358]), torch.Size([137358]))

In [ ]:
loader = DataLoader(list(zip(X, y)), batch_size=64, num_workers=4)

In [ ]:
def words_sim(indices):
  for ind in indices:
    word_emb = skipgram.emb(ind).reshape(1, -1)
    cos_sim = word_emb @ skipgram.emb.weight.T / \
              ((word_emb**2).sum(axis=1)**0.5 * (skipgram.emb.weight**2).sum(axis=1)**0.5)
    top = th.argsort(cos_sim[0][1:], descending=True)[:5] + 1
    top_words = dataset.vocab.lookup_tokens(list(top))
    print(f'{top_words[0]}\t-> {top_words[1:]}')

In [ ]:
n_epoch = 10
lr = 0.1
skipgram = SkipGramModel(len(dataset.vocab))
crit = nn.CrossEntropyLoss()
optimizer = optim.Adam(skipgram.parameters(), lr=lr)
rand_ind = th.randint(len(dataset.vocab), size=(3,))

for epoch in range(n_epoch):
  # for X_b, y_b in loader:
  out = skipgram(X)
  loss = crit(out, y)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'{epoch=} {loss.item()=}')
  words_sim(rand_ind)

epoch=0 loss.item()=6.19916296005249
очередь	-> ['переговоров', 'возможно', 'отказался', 'стало']
сегодня	-> ['стран', 'решения', 'ходе', 'зрения']
день	-> ['условиях', 'накануне', 'идет', 'долларов']
epoch=1 loss.item()=10.549703598022461
очередь	-> ['переговоров', 'отказался', 'возможно', 'края']
сегодня	-> ['стран', 'решения', 'ходе', 'источник']
день	-> ['условиях', 'накануне', 'идет', 'банки']
epoch=2 loss.item()=13.772513389587402
очередь	-> ['переговоров', 'отказался', 'возможно', 'края']
сегодня	-> ['стран', 'решения', 'ходе', 'зрения']
день	-> ['условиях', 'накануне', 'банки', 'идет']
epoch=3 loss.item()=13.631192207336426
очередь	-> ['переговоров', 'отказался', 'возможно', 'края']
сегодня	-> ['стран', 'ходе', 'решения', 'зрения']
день	-> ['условиях', 'накануне', 'банки', 'идет']
epoch=4 loss.item()=13.706318855285645
очередь	-> ['переговоров', 'отказался', 'края', 'возможно']
сегодня	-> ['стран', 'ходе', 'решения', 'зрения']
день	-> ['условиях', 'банки', 'накануне', 'долларов

<p class="task" id="5"></p>

5\. Реализуйте класс `NegativeSampler`, который позволяет сгенерировать набор отрицательных примеров. Для генерации отрицательных примеров выбирайте токены пропорционально величине $C(w)^{\frac{3}{4}}$, где $C(w)$ - частота токена в корпусе.


- [ ] Проверено на семинаре

In [ ]:
class NegativeSampler:
    def __init__(self, corpus):
        self.corpus = corpus
        self.token_freq = self.calc_token_freq()

    def calc_token_freq(self):
        token_freq = nltk.FreqDist(self.corpus)
        return {token: freq**0.75 for token, freq in token_freq.items()}

    def generate_negative(self, num_examples):
        prob_dist = nltk.probability.DictionaryProbDist(self.token_freq, normalize=True)
        negative_examples = [prob_dist.generate() for _ in range(num_examples)]
        return negative_examples

corpus = corpus_t[0]
sampler = NegativeSampler(corpus_t[0])
negative_examples = sampler.generate_negative(4)
negative_examples

['такое', 'администрации', 'сообщили', 'рф']

In [ ]:
corpus_t[0]

['президент',
 'россии',
 'владимир',
 'путин',
 'считает',
 'концепция',
 'реформирования',
 'армии',
 'должна',
 'готова',
 'ноябрю',
 'такое',
 'заявление',
 'сделал',
 'совещанием',
 'членов',
 'совета',
 'безопасности',
 'рф',
 'сообщили',
 'рбк',
 'администрации',
 'президента',
 'путин',
 'также',
 'отметил',
 'эта',
 'реформа',
 'должна',
 'проводиться',
 'учетом',
 'проблем',
 'существующих',
 'настоящее',
 'время',
 'вооруженных',
 'силах',
 'рф']

<p class="task" id="6"></p>

6\. Реализуйте и настройте skip-gram модель с использованием negative sampling. Перед началом обучения выберите случайным образом несколько слов из датасета и для каждого из них выведите на экран 3 ближайших слова в смысле косинусной близости между эмбеддингами. В процессе настройки для валидации периодически выводите на экран информацию о ближайших словах для этих слов. Выведите на экран график значения функции потерь в зависимости от номера эпохи.  

- [ ] Проверено на семинаре

## Обратная связь
- [ ] Хочу получить обратную связь по решению